In [12]:
# https://stackoverflow.com/questions/47805170/whats-the-hardware-spec-for-google-colaboratory

In [1]:
!df -h

"df" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 45
model name	: Intel(R) Core(TM) i7-3930K CPU @ 3.20GHz
stepping	: 7
cpu MHz		: 4000.000
cache size	: 256 KB
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush acpi mmx fxsr sse sse2 ss ht tm pbe pni est tm2 ssse3 cx16 xtpr pdcm sse4_1 sse4_2 popcnt aes xsave osxsave avx hypervisor lahf_lm ida epb xsaveopt pln pts dtherm
TLB size	: 0 4K pages
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6
model		: 45
model name	: Intel(R) Core(TM) i7-3930K CPU @ 3.20GHz
stepping	: 7
cpu MHz		: 4000.000
cache size	: 256 KB
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush acpi mmx fxsr sse sse2 ss ht tm pbe pni est tm2 ssse3 cx16 xtpr pdc

In [3]:
!cat /proc/meminfo

MemTotal:       50268220 kB
MemFree:        36185176 kB
HighTotal:             0 kB
HighFree:              0 kB
LowTotal:       50268220 kB
LowFree:        36185176 kB
SwapTotal:        138680 kB
SwapFree:          23640 kB


# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [46]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

Спарсим текст из файлика.\
**`Requests`** [tutorial](https://realpython.com/python-requests/)

In [47]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [48]:
text = text.lower()

Разобьём весь текст на предложения. \
**`re`** [tutorial](https://tproger.ru/translations/regular-expression-python/)

In [49]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

**`nltk`** [tutorial](https://www.guru99.com/nltk-tutorial.html)

In [8]:
import nltk

# nltk.download('all') # если хотим всё и сразу 

# нам хватит вот этого: 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ksetd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ksetd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [51]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [52]:
from nltk.tokenize import word_tokenize

print(word_tokenize(sents[0]))
sents[:2]

['преступление', 'и', 'наказание', 'роман', 'в', 'шести', 'частях', 'с', 'эпилогом', 'часть', 'первая', 'i', 'в', 'начале', 'июля', ',', 'в', 'чрезвычайно', 'жаркое', 'время', ',', 'под', 'вечер', ',', 'один', 'молодой', 'человек', 'вышел', 'из', 'своей', 'каморки', ',', 'которую', 'нанимал', 'от', 'жильцов', 'в', 'с', '-', 'м', 'переулке', ',', 'на', 'улицу', 'и', 'медленно', ',', 'как', 'бы', 'в', 'нерешимости', ',', 'отправился', 'к', 'к', '-', 'ну', 'мосту', '.']


['преступление и наказание роман  в шести частях с эпилогом  часть первая  i  в начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в с - м переулке, на улицу и медленно, как бы в нерешимости, отправился к к - ну мосту.',
 'он благополучно избегнул встречи с своею хозяйкой на лестнице.']

In [53]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [54]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]
sents_tokenize[:2]

[['преступление',
  'и',
  'наказание',
  'роман',
  'в',
  'шести',
  'частях',
  'с',
  'эпилогом',
  'часть',
  'первая',
  'i',
  'в',
  'начале',
  'июля',
  'в',
  'чрезвычайно',
  'жаркое',
  'время',
  'под',
  'вечер',
  'один',
  'молодой',
  'человек',
  'вышел',
  'из',
  'своей',
  'каморки',
  'которую',
  'нанимал',
  'от',
  'жильцов',
  'в',
  'с',
  'м',
  'переулке',
  'на',
  'улицу',
  'и',
  'медленно',
  'как',
  'бы',
  'в',
  'нерешимости',
  'отправился',
  'к',
  'к',
  'ну',
  'мосту'],
 ['он',
  'благополучно',
  'избегнул',
  'встречи',
  'с',
  'своею',
  'хозяйкой',
  'на',
  'лестнице']]

In [55]:
# Flatten без numpy :) 
words = [item for sent in sents_tokenize for item in sent]
words[:10]

['преступление',
 'и',
 'наказание',
 'роман',
 'в',
 'шести',
 'частях',
 'с',
 'эпилогом',
 'часть']

In [56]:
len(words) # всего слов

173403

In [57]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [58]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [59]:
len(stopwords_ru)

151

In [60]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                        for sent in sents_tokenize]
sents_tokenize[:2]

[['преступление',
  'наказание',
  'роман',
  'шести',
  'частях',
  'эпилогом',
  'часть',
  'первая',
  'i',
  'начале',
  'июля',
  'чрезвычайно',
  'жаркое',
  'время',
  'вечер',
  'молодой',
  'человек',
  'вышел',
  'своей',
  'каморки',
  'которую',
  'нанимал',
  'жильцов',
  'м',
  'переулке',
  'улицу',
  'медленно',
  'нерешимости',
  'отправился',
  'мосту'],
 ['благополучно', 'избегнул', 'встречи', 'своею', 'хозяйкой', 'лестнице']]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [20]:
!pip install pymorphy2

In [61]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авиньон и пленил пап!"
tokens = tokenizer.tokenize(text)  # regexp tokenizer initialized earlier

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авиньон и пленить папа'

In [62]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Зачем нужно возвращать словарь в такой конструкции: `morph.normal_forms(token)`? Ответ ниже.

In [63]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета. 

In [64]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [70]:
sents_tokenize[0][:10]

['преступление',
 'наказание',
 'роман',
 'шесть',
 'часть',
 'эпилог',
 'часть',
 'первый',
 'i',
 'начало']

In [65]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [66]:
len(words) # всего слов

93069

In [67]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочется больше предобработки, хороший мануал по [ссылке](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb).  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [68]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [69]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.


**`gensim`** [documentation](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py)\
[Нововведения](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4) в **gensim 4.0**


In [30]:
!pip install gensim==4.1.2

In [71]:
#%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

(0, 0)

Другой способ инициализировать и тренировать модель.

Такой синтаксис под капотом точно так же строит словарь и вызывает метод `train`

In [72]:
model2 = Word2Vec(sentences=sents_tokenize, vector_size=100, window=2, min_count=3, workers=-1, epochs=100)
print(model2)

Word2Vec(vocab=4768, vector_size=100, alpha=0.025)


In [73]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [74]:
len(model.wv), len(model2.wv)

(4768, 4768)

In [35]:
# model.wv.key_to_index

In [75]:
'старуха' in model.wv.key_to_index, 'старуха' in model2.wv.key_to_index

(True, True)

In [76]:
model.wv.get_vecattr("старуха", "count"), model2.wv.get_vecattr("старуха", "count") 

(72, 72)

## 3. Свойства модели

In [77]:
# вектор слова
model.wv['старуха'][:10], model2.wv['старуха'][:10]

(array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
         0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ],
       dtype=float32),
 array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
         0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ],
       dtype=float32))

In [78]:
# размерность вектора
model.wv['старуха'].shape, model2.wv['старуха'].shape

((100,), (100,))

In [79]:
# похожести слов 
model.wv.similarity('тварь', 'право'), model2.wv.similarity('тварь', 'право')

(-0.03793418, -0.03793418)

In [82]:
# самые похожие
model.wv.most_similar('топор', topn=10)

[('выехать', 0.3434045910835266),
 ('вицмундир', 0.3418233096599579),
 ('нравиться', 0.3186732232570648),
 ('капернаум', 0.3163142502307892),
 ('преступник', 0.3118082582950592),
 ('случайность', 0.29301321506500244),
 ('поворотить', 0.2834480404853821),
 ('обыкновение', 0.2801940143108368),
 ('буля', 0.27475351095199585),
 ('поручить', 0.2660590708255768)]

In [83]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

[('съехать', 0.3254740536212921),
 ('раздражать', 0.31844109296798706),
 ('линия', 0.31684327125549316),
 ('покоситься', 0.30329668521881104),
 ('детски', 0.29883092641830444),
 ('чашка', 0.29052451252937317),
 ('вырываться', 0.2874678075313568),
 ('влево', 0.2853025794029236),
 ('вызывать', 0.2818695306777954),
 ('холодно', 0.28120535612106323)]

### KeyedVectors

**TL;DR:** the main difference is that KeyedVectors do not support further training. On the other hand, by shedding the internal data structures necessary for training, KeyedVectors offer a smaller RAM footprint and a simpler interface.

[Source](https://radimrehurek.com/gensim/models/keyedvectors.html#)

In [84]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = wv['раскольников']  # Get numpy vector of a word
vector.shape, vector

((100,),
 array([ 0.0006687 ,  0.00648878,  0.00587001, -0.00017738,  0.0023801 ,
        -0.00651387, -0.00936036, -0.00494762, -0.00538789,  0.00058702,
        -0.00396823, -0.00036959,  0.00763455,  0.00973569,  0.00787588,
         0.00238922, -0.00271333,  0.00243172, -0.00268992, -0.00895014,
         0.00689304, -0.00864854,  0.00378768, -0.00395623, -0.00554306,
        -0.00104358, -0.00656298,  0.00858411, -0.00687794, -0.00896055,
         0.00576626,  0.00446401,  0.00473145,  0.00242694, -0.00372551,
        -0.0051173 ,  0.00177215,  0.00834657, -0.00565228, -0.00238244,
         0.00184836,  0.00672219, -0.0063974 , -0.00452775, -0.00263857,
         0.00133483,  0.00537128,  0.00554937, -0.0087095 ,  0.00727261,
        -0.00250222, -0.00284264, -0.00885856, -0.00577508,  0.00944861,
         0.00308753, -0.00804277,  0.0028074 , -0.00707276, -0.00905789,
         0.00386578,  0.00203697,  0.00125465,  0.00884633,  0.0062607 ,
        -0.00507063,  0.00272783,  0.00254

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [85]:
model_path = "./our_w2v.model"
model.save(model_path)

In [86]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [87]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [ ]:
# А теперь ваш код предобработки


In [ ]:
sents_tokenize2[10]

In [ ]:
len(sents_tokenize2)

Дополняем модель.

In [ ]:
# И теперь ваш код обучения

In [ ]:
'ядро' in model.wv.vocab

In [ ]:
'ядро' in our_model.wv.vocab

In [ ]:
our_model.wv.most_similar('ядро')

Пример со старым словом.

In [ ]:
our_model.wv.most_similar('сын')

In [ ]:
model.wv.most_similar('сын')